In [1]:
import c3d
import mat4py
import functools
import scipy.io
from torch.utils.data import TensorDataset, DataLoader, Dataset
from tqdm import notebook
from torch.optim import Adam
from torch.utils.data import DataLoader
import tqdm
import math
#from torch.utils.tensorboard import SummaryWriter
import matplotlib.pyplot as plt
device ='cpu'
import numpy as np
import torch
from torch_geometric.data import Data
from sklearn.decomposition import PCA
from torch.autograd.functional import jacobian, hessian
torch.set_default_dtype(torch.double)
from ipynb.fs.defs.gBAOAB import gBAOAB_integrator, gBAOAB_integrator_mass, gBAOAB_step_exact
from ipywidgets import IntProgress
from IPython.display import display
import time
import random
from torchmin import minimize

In [2]:
mat = scipy.io.loadmat('C:\\Users\\Desktop\\Thesis\\HumanEval\\S2\\Mocap_Data\\Walking_1.mat')

In [3]:
mat['Markers']

array([[[-431.09890747,  323.9395752 ,  995.49688721],
        [-198.52206421,  280.0027771 , 1094.01745605],
        [-270.77734375,  195.53082275, 1024.84570312],
        ...,
        [   0.        ,    0.        ,    0.        ],
        [   0.        ,    0.        ,    0.        ],
        [   0.        ,    0.        ,    0.        ]],

       [[-434.81082153,  322.97921753,  996.29925537],
        [-199.78430176,  281.39175415, 1095.79394531],
        [-268.5760498 ,  195.46652222, 1025.36669922],
        ...,
        [   0.        ,    0.        ,    0.        ],
        [   0.        ,    0.        ,    0.        ],
        [   0.        ,    0.        ,    0.        ]],

       [[-438.90246582,  321.10977173,  997.38763428],
        [-201.00534058,  282.6975708 , 1097.75732422],
        [-266.43099976,  195.89286804, 1026.08081055],
        ...,
        [   0.        ,    0.        ,    0.        ],
        [   0.        ,    0.        ,    0.        ],
        [   0.        

In [4]:
mat['ParameterGroup'][0][2][2][0][10][2][0][35]

array(['Clavicle'], dtype='<U8')

In [5]:
mat['Markers'][1806][38]

array([-506.78799438,  -51.61029434,  805.42022705])

In [28]:
joints = [0,1,3,5,8,10,12,14,15,17,25,27,32,35,22,23]
indices = range(0,39)
for i in range(len(indices)):
    print(f"{i}: {mat['ParameterGroup'][0][2][2][0][10][2][0][i]} ")

0: ['Left ASIS'] 
1: ['Left elbow'] 
2: ['Left forearm (optional alternative anti-sym)'] 
3: ['Left wrist bar pinkie side (alternative pair 1)'] 
4: ['Left wrist bar thumb side (alternative pair 1)'] 
5: ['Right ASIS'] 
6: ['Left finger'] 
7: ['Upper back'] 
8: ['Right elbow'] 
9: ['Right upper arm (optional alternative anti-sym)'] 
10: ['Right wrist bar pinkie side (alternative pair 1)'] 
11: ['Right finger'] 
12: ['Left knee'] 
13: ['Right back (optional alternative anti-sym)'] 
14: ['Left shoulder'] 
15: ['Right knee'] 
16: ['Back of neck'] 
17: ['Right shoulder'] 
18: ['Right tibial wand marker'] 
19: ['Left tibial wand marker'] 
20: ['Left back head'] 
21: ['Right back head'] 
22: ['Left front head'] 
23: ['Right front head'] 
24: ['Left heel'] 
25: ['Left ankle'] 
26: ['Left toe (2nd metatarsel head)'] 
27: ['Right ankle'] 
28: ['Right heel'] 
29: ['Right toe (2nd metatarsel head)'] 
30: ['Right PSIS'] 
31: ['Right wrist bar thumb side (alternative pair 1)'] 
32: ['Sternum'] 
33:

In [29]:
# note - I should sort these
#indices = range(0,39)
# transforming
transform = np.zeros((39,3))
for i in range(39):
    transform[i][2] = 1

train_dataset = []
for i in range(len(mat['Markers'])//2):
  data = [mat['Markers'][i][j] for j in indices]
  train_dataset.append(torch.tensor(np.array(data)+transform, dtype =torch.double).unsqueeze(0))

In [30]:
test_dataset = []

for i in range(len(mat['Markers'])//2, len(mat['Markers'])):
  data = [mat['Markers'][i][j] for j in indices]
  test_dataset.append(torch.tensor(np.array(data)+transform, dtype =torch.double).unsqueeze(0))

In [31]:
train_data = np.array([d.numpy() for d in train_dataset])
test_data = np.array([d.numpy() for d in test_dataset])

## PCA

In [32]:
list = np.transpose(np.array([data[0].flatten().numpy() for data in train_dataset]))

# # Create a PCA object
# pca = PCA()  # Specify the number of components you want to retain
#
# pca_result = pca.fit_transform(list)
# eigenposes = np.transpose(pca.components_)
# eigenvalues = pca.explained_variance_

In [33]:
train_dataset[0].size()

torch.Size([1, 39, 3])

In [34]:
np.shape(list)

(117, 948)

In [35]:
U, S,_=np.linalg.svd(list)

In [36]:
eigenposes = np.transpose(U)
eigenvalues = S

## Projections

In [37]:
train_data = np.array([d.numpy() for d in train_dataset])

In [38]:
train_data.max()

1691.377197265625

In [39]:
train_data.min()

-1145.9652099609375

In [40]:
# getting the average pose
avg_pose = np.squeeze(train_data.mean(axis=0))
avg_pose = np.ndarray.flatten(avg_pose)

In [41]:
avg_pose.reshape(39,3)

array([[-1.05234848e+01, -9.75135418e+01,  9.93005519e+02],
       [ 1.81891002e+01, -8.01991132e+01,  1.09734191e+03],
       [ 2.06319369e+01, -9.40360743e+01,  1.03622893e+03],
       [ 2.44225704e+01, -8.96347777e+01,  8.66264727e+02],
       [ 1.11231989e+01, -1.05407768e+02,  9.06995790e+02],
       [-3.30211723e+01, -8.83074704e+01,  1.01229484e+03],
       [ 1.77338899e+01, -1.01651113e+02,  8.43746180e+02],
       [-5.84129434e+00, -6.09202360e+01,  1.25282430e+03],
       [-4.27508576e+01, -5.27883308e+01,  1.10323775e+03],
       [-4.70981177e+01, -6.13500123e+01,  1.22599984e+03],
       [-6.50190309e+01, -6.44388580e+01,  8.63607886e+02],
       [-6.71726576e+01, -7.51322394e+01,  8.43437988e+02],
       [-3.45383179e+00, -8.61751637e+01,  5.16346356e+02],
       [-1.50537029e+01, -5.69759235e+01,  1.38266730e+03],
       [ 9.30007439e+00, -9.08573692e+01,  1.46598217e+03],
       [-3.71105452e+01, -6.75618415e+01,  5.16885675e+02],
       [-1.08104832e+01, -7.26948328e+01

In [42]:
def length_constraint(i,j, init):
    def constraint_fn(x):
        return (x[3*i]- x[3*j])**2 +(x[3*i+1]- x[3*j+1])**2 + (x[3*i+2]- x[3*j+2])**2 - ((init[3*i]- init[3*j])**2 +(init[3*i+1]- init[3*j+1])**2 + (init[3*i+2]- init[3*j+2])**2)
    return constraint_fn

In [43]:
for i in range(len(joints)):
    print(f"{i}: {mat['ParameterGroup'][0][2][2][0][10][2][0][joints[i]]} ")

0: ['Left ASIS'] 
1: ['Left elbow'] 
2: ['Left wrist bar pinkie side (alternative pair 1)'] 
3: ['Right ASIS'] 
4: ['Right elbow'] 
5: ['Right wrist bar pinkie side (alternative pair 1)'] 
6: ['Left knee'] 
7: ['Left shoulder'] 
8: ['Right knee'] 
9: ['Right shoulder'] 
10: ['Left ankle'] 
11: ['Right ankle'] 
12: ['Sternum'] 
13: ['Clavicle'] 
14: ['Left front head'] 
15: ['Right front head'] 


In [44]:
from numpy import linalg as la

In [45]:
def isPD(B):
    """Returns true when input is positive-definite, via Cholesky"""
    try:
        _ = la.cholesky(B)
        return True
    except la.LinAlgError:
        return False

def nearestPD(A):
    """Find the nearest positive-definite matrix to input
    A Python/Numpy port of John D'Errico's `nearestSPD` MATLAB code [1], which
    credits [2].
    [1] https://www.mathworks.com/matlabcentral/fileexchange/42885-nearestspd
    [2] N.J. Higham, "Computing a nearest symmetric positive semidefinite
    matrix" (1988): https://doi.org/10.1016/0024-3795(88)90223-6
    """

    B = (A + A.T) / 2
    _, s, V = la.svd(B)

    H = np.dot(V.T, np.dot(np.diag(s), V))

    A2 = (B + H) / 2

    A3 = (A2 + A2.T) / 2

    if isPD(A3):
        return A3

    spacing = np.spacing(la.norm(A))
    # The above is different from [1]. It appears that MATLAB's `chol` Cholesky
    # decomposition will accept matrixes with exactly 0-eigenvalue, whereas
    # Numpy's will not. So where [1] uses `eps(mineig)` (where `eps` is Matlab
    # for `np.spacing`), we use the above definition. CAVEAT: our `spacing`
    # will be much larger than [1]'s `eps(mineig)`, since `mineig` is usually on
    # the order of 1e-16, and `eps(1e-16)` is on the order of 1e-34, whereas
    # `spacing` will, for Gaussian random matrixes of small dimension, be on
    # othe order of 1e-16. In practice, both ways converge, as the unit test
    # below suggests.
    I = np.eye(A.shape[0])
    k = 1
    while not isPD(A3):
        mineig = np.min(np.real(la.eigvals(A3)))
        A3 += I * (-mineig * k**2 + spacing)
        k += 1

    return A3

# note - using NearPD from Higham

## Running Evaluation Loop

In [46]:
def avg_j2j(frame1,frame2):
    dists = []
    frame_1 = np.array(frame1)
    frame_2 = np.array(frame2)
    for i in range(len(joints)):
        d = math.dist(frame_1[joints[i]], frame_2[joints[i]])
        dists.append(d)
    return np.mean(dists)

In [47]:
def blank(x):
    t = x[0]
    return torch.tensor([0.0])

In [48]:
h = 0.1

In [50]:
diffs = []

f = IntProgress(min=0, max=len(test_dataset[0:100]))
display(f)
for frame1 in test_dataset[0:100]:

    frame = torch.squeeze(frame1)

    M = torch.eye(39*3)


    # initialising from a random training pose
    x_init = torch.flatten(train_dataset[random.randint(0,len(train_dataset))])
    v_init = torch.zeros_like(x_init)

    # defining the constraints
    #wrist_elbow_l = length_constraint(joints[1],joints[2],x_init)
    wrist_elbow_l = length_constraint(joints[1],joints[2],x_init)
    elbow_shoulder_l = length_constraint(joints[1],joints[7],x_init)
    wrist_elbow_r = length_constraint(joints[5],joints[4],x_init)
    elbow_shoulder_r= length_constraint(joints[4],joints[9],x_init)

    knee_ankle_l = length_constraint(joints[6],joints[10],x_init)
    knee_ankle_r = length_constraint(joints[8],joints[11],x_init)

    knee_asis_l = length_constraint(joints[6],joints[0],x_init)
    knee_asis_r = length_constraint(joints[8],joints[3],x_init)

    shoulder_clav_l = length_constraint(joints[13],joints[7],x_init)
    shoulder_clav_r = length_constraint(joints[13],joints[9],x_init)

    #stern_clav_l = length_constraint(joints[12],joints[13],x_init)

    gs = [wrist_elbow_l,wrist_elbow_r,elbow_shoulder_l,elbow_shoulder_r,knee_ankle_l, knee_ankle_r,shoulder_clav_l,shoulder_clav_r] #,knee_asis_l, knee_asis_r] #,shoulder_clav_l,shoulder_clav_r,stern_clav_l]

    # defining and randomly perturbing the image
    fx = 1.0
    fy = 1.0
    cx = 0
    cy = 0
    a1 = torch.tensor([[fx], [0], [cx]], dtype=torch.double)
    a2 = torch.tensor([[0], [fy], [cy]], dtype=torch.double)
    a3 = torch.tensor([[0],[0],[1]], dtype=torch.double)
    i1 = frame @ a1
    i2 = frame @ a2
    i3 = frame @ a3

    image = torch.hstack([i1/i3, i2/i3])


    #image += torch.randn_like(image)* (torch.randint(11, size = (1,)))

    # need to re-define density, force and mass to use the new image
    def neg_log_density(pose2):
        pose = pose2.reshape((39,3))

        transform = torch.zeros((39,3))
        for i in range(39):
            transform[i][2] = 1

        pose = torch.squeeze(pose) + transform
        fx = 1.0
        fy = 1.0
        cx = 0
        cy = 0
        a1 = torch.tensor([[fx], [0], [cx]], dtype=torch.double)
        a2 = torch.tensor([[0], [fy], [cy]], dtype=torch.double)
        a3 = torch.tensor([[0],[0],[1]], dtype=torch.double)

        # Map the 3D point to 2D point
        p1 = pose@ a1
        p2 = pose @ a2
        p3 = pose @ a3

        projection = torch.hstack([p1/p3, p2/p3])

        n = projection - image
        #print(n)
        # note - taking the variance to be 1...
        pi = torch.tensor([0.], requires_grad=True)
        for point in n:
            pi = pi+ torch.linalg.norm(point)**2 / 36.36

        p2 = torch.flatten(pose)
        for i in range(len(eigenvalues)):
            pi = pi+ (torch.dot(torch.tensor(eigenposes[i]),(p2-torch.tensor(avg_pose)))**2 / eigenvalues[i])
        return pi

    def force(x):
        #print("x = ",x)
        #print("force = ",torch.squeeze(jacobian(neg_log_density, x), dim = 0))
        return torch.squeeze(jacobian(neg_log_density, x), dim = 0)


    positions, velocities = gBAOAB_integrator(x_init,v_init, force,gs, h,M, .001, 1, 500)

    # getting the mean position
    mean = torch.tensor(np.array(torch.stack(positions[100:])).mean(axis=0))

    # getting mean difference between this and the pose
    jd = avg_j2j(mean.reshape((39,3)),frame)
    print("GBAOAB: ", jd)
    diffs.append(jd)

    # BFGS
    result = minimize(neg_log_density, x_init, method='bfgs')
    jd2 = avg_j2j(result.x.reshape((39,3)),frame)
    print("BFGS: ", jd2)

    print(f"Average joint movement: {mean.mean()}")

IntProgress(value=0)


KeyboardInterrupt



In [ ]:
velocities_1 = [vel[0] for vel in velocities]

In [ ]:
forces_1 = [force(x)[0] for x in positions]

In [ ]:
plt.plot(velocities_1, label = "Velocities")
plt.plot(forces_1, label = "Force")
plt.legend()

In [ ]:
velocities_1

In [ ]:
x_init.reshape((39,3))

In [ ]:
# defining and randomly perturbing the image
fx = 1.0
fy = 1.0
cx = 0
cy = 0
a1 = torch.tensor([[fx], [0], [cx]], dtype=torch.double)
a2 = torch.tensor([[0], [fy], [cy]], dtype=torch.double)
a3 = torch.tensor([[0],[0],[1]], dtype=torch.double)
i1 = frame @ a1
i2 = frame @ a2
i3 = frame @ a3

image = torch.hstack([i1/i3, i2/i3])


image += torch.randn_like(image)* (torch.randint(11, size = (1,)))

In [ ]:
x_init

In [ ]:
def test_initial(x):
        pose = x.reshape((39,3))

        transform = torch.zeros((39,3))
        for i in range(39):
            transform[i][2] = 1

        pose = torch.squeeze(pose) + transform

        fx = 1.0
        fy = 1.0
        cx = 0
        cy = 0
        a1 = torch.tensor([[fx], [0], [cx]], dtype=torch.double)
        a2 = torch.tensor([[0], [fy], [cy]], dtype=torch.double)
        a3 = torch.tensor([[0],[0],[1]], dtype=torch.double)

        # Map the 3D point to 2D point
        p1 = pose@ a1
        p2 = pose @ a2
        p3 = pose @ a3

        projection = torch.hstack([p1/p3, p2/p3])

        n = projection - image
        print(n)
        # note - taking the variance to be 1...
        pi = torch.tensor([0.], requires_grad=True)
        for point in n:
            pi = pi+ torch.linalg.norm(point)**2
        return pi

In [ ]:
image

In [ ]:
jacobian(test_initial, x_init)[0]

In [ ]:
def testing2(x):
    '''computing the sum over i of eigenpose[i]^T (x - avg_pose)^2 / eigenval[i]'''
    pi = torch.tensor([0.0], requires_grad=True)
    for i in range(len(eigenvalues)):
        pi = pi+ (torch.dot(torch.tensor(eigenposes[i]),(x-torch.tensor(avg_pose)))**2 / eigenvalues[i])
    return pi

In [ ]:
def deriv_test(x):
    j =0
    deriv = 2*eigenposes[j][0]*(2*x[0] - 2* avg_pose[0])
    for i in range(1,len(x)):
        deriv += eigenposes[j][i]*(2*x[i] - 2* avg_pose[i])
    return deriv

In [ ]:
testing2(x_init)

In [ ]:
jacobian(testing2, x_init)[0][0]

In [ ]:
deriv_test(x_init)

In [ ]:
#math.dist(positions[0].reshape((39,3))[1],positions[0].reshape((39,3))[2])

In [ ]:
second = [torch.tensor([pos[9], pos[10],pos[11]]) for pos in positions]

In [ ]:
x_init = torch.flatten(train_dataset[random.randint(0,len(train_dataset))]).reshape((39,3))

In [ ]:
x_init.reshape((39,3))

In [ ]:
positions[199].reshape((39,3))

In [ ]:
frame

In [ ]:
pose = positions[199].reshape((39,3))
pose = torch.squeeze(pose) + transform
fx = 1.0
fy = 1.0
cx = 0
cy = 0
a1 = torch.tensor([[fx], [0], [cx]], dtype=torch.double)
a2 = torch.tensor([[0], [fy], [cy]], dtype=torch.double)
a3 = torch.tensor([[0],[0],[1]], dtype=torch.double)

# Map the 3D point to 2D point
p1 = pose@ a1
p2 = pose @ a2
p3 = pose @ a3

projection = torch.hstack([p1/p3, p2/p3])

In [ ]:
projection

In [ ]:
frame1 = torch.squeeze(frame1)
# Map the 3D point to 2D point
p1 = frame1 @ a1
p2 = frame1 @ a2
p3 = frame1 @ a3

projection = torch.hstack([p1/p3, p2/p3])

In [ ]:
projection

## Using different masses

In [ ]:
h = .5

In [ ]:
plt.figure()
i = 0
std =1
diffs = []
diffs1 = []
diffs_bfgs = []
f = IntProgress(min=0, max=len(test_dataset[0:100]))
display(f)
for frame1 in test_dataset[0:100]:
    i +=1

    frame = torch.squeeze(frame1)
    #print("Target: ",frame.reshape(39,3))

    M = torch.eye(39*3)



    # initialising from a random training pose
    x_init = torch.flatten(train_dataset[random.randint(0,len(train_dataset))-1])
    #print("Init: ",x_init.reshape(39,3))
    v_init = torch.zeros_like(x_init)

    # defining the constraints
    #wrist_elbow_l = length_constraint(joints[1],joints[2],x_init)
    wrist_elbow_l = length_constraint(joints[1],joints[2],x_init)
    elbow_shoulder_l = length_constraint(joints[1],joints[7],x_init)
    wrist_elbow_r = length_constraint(joints[5],joints[4],x_init)
    elbow_shoulder_r= length_constraint(joints[4],joints[9],x_init)

    knee_ankle_l = length_constraint(joints[6],joints[10],x_init)
    knee_ankle_r = length_constraint(joints[8],joints[11],x_init)

    knee_asis_l = length_constraint(joints[6],joints[0],x_init)
    knee_asis_r = length_constraint(joints[8],joints[3],x_init)

    shoulder_clav_l = length_constraint(joints[13],joints[7],x_init)
    shoulder_clav_r = length_constraint(joints[13],joints[9],x_init)

    #stern_clav_l = length_constraint(joints[12],joints[13],x_init)

    gs = [wrist_elbow_l,wrist_elbow_r,elbow_shoulder_l,elbow_shoulder_r,knee_ankle_l, knee_ankle_r]#,shoulder_clav_l,shoulder_clav_r] #,knee_asis_l, knee_asis_r] #,shoulder_clav_l,shoulder_clav_r,stern_clav_l]

    # defining and randomly perturbing the image
    fx = 1.0
    fy = 1.0
    cx = 0
    cy = 0
    a1 = torch.tensor([[fx], [0], [cx]], dtype=torch.double)
    a2 = torch.tensor([[0], [fy], [cy]], dtype=torch.double)
    a3 = torch.tensor([[0],[0],[1]], dtype=torch.double)
    i1 = frame @ a1
    i2 = frame @ a2
    i3 = frame @ a3

    image = torch.hstack([i1/i3, i2/i3])


    image += torch.randn_like(image)*std

    # need to re-define density, force and mass to use the new image
    def neg_log_density(pose2):
        pose = pose2.reshape((39,3))

        transform = torch.zeros((39,3))
        for i in range(39):
            transform[i][2] = 1

        pose = torch.squeeze(pose) + transform
        fx = 1.0
        fy = 1.0
        cx = 0
        cy = 0
        a1 = torch.tensor([[fx], [0], [cx]], dtype=torch.double)
        a2 = torch.tensor([[0], [fy], [cy]], dtype=torch.double)
        a3 = torch.tensor([[0],[0],[1]], dtype=torch.double)

        # Map the 3D point to 2D point
        p1 = pose@ a1
        p2 = pose @ a2
        p3 = pose @ a3

        projection = torch.hstack([p1/p3, p2/p3])

        n = projection - image
        #print(n)
        # note - taking the variance to be 1...
        pi = torch.tensor([0.], requires_grad=True)
        for point in n:
            pi = pi+ torch.linalg.norm(point)**2 / std

        p2 = torch.flatten(pose)
        for i in range(len(eigenvalues)):
            pi = pi+ (torch.dot(torch.tensor(eigenposes[i]),(p2-torch.tensor(avg_pose)))**2 / eigenvalues[i]**2)
        return pi

    def force(x):
        #print("x = ",x)
        #print("force = ",torch.squeeze(jacobian(neg_log_density, x), dim = 0))
        return torch.tensor([1.])*torch.squeeze(jacobian(neg_log_density, x), dim = 0)

    def mass(x):
        #print(torch.tensor(nearestPD(hessian(neg_log_density, x).numpy())).mean())
        return torch.tensor(nearestPD(hessian(neg_log_density, x).numpy()))

    #positions, velocities = gBAOAB_integrator_mass(x_init,v_init, force,gs, h,mass, 1, 1,200)
    positions, velocities = gBAOAB_integrator(x_init,v_init, force,gs, h,M, 1, 1, 100)

    densities = [neg_log_density(p).detach().numpy() for p in positions]
    plt.plot(densities,label = f"Frame {i}")
    plt.show()
    forces = [force(p) for p in positions]
    # getting the mean position
    mean = torch.tensor(np.array(torch.stack(positions[50:])).mean(axis=0))
    #mean1 =torch.tensor(np.array(torch.stack(positions1[50:])).mean(axis=0))
    # getting mean difference between this and the pose
    jd = avg_j2j(mean.reshape((39,3)),frame)
    print("GBAOAB (varied mass): ", jd)
    diffs.append(jd)
    #jd1 = avg_j2j(mean1.reshape((39,3)),frame)
    #print("GBAOAB (constant mass): ", jd1)
    #diffs1.append(jd1)

    # BFGS
    #result = minimize(neg_log_density, x_init, method='bfgs')
    #jd2 = avg_j2j(result.x.reshape((39,3)),frame)
    #print("BFGS: ", jd2)
    diffs_bfgs.append(jd2)
    print(f"Average joint movement: {avg_j2j(mean.reshape((39,3)), x_init.detach().reshape((39,3)))}")
    #print(f"Average joint movement - constant mass: {mean1.mean()}")

IntProgress(value=0)

In [ ]:
densities

In [ ]:
forces1 = [torch.linalg.norm(f) for f in forces]
forces1

In [ ]:
forces[-1]

In [ ]:
forces[0]